<a href="https://colab.research.google.com/github/GowthamNats/Recipe-Recommender-System/blob/main/SNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import pandas as pd

import os
df = pd.read_json('../content/test.json')
pf = df
df.ingredients = df.ingredients.apply(lambda ings : [ing.lower() for ing in ings])
df

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."
...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
9941,22339,"[black pepper, salt, parmigiano reggiano chees..."
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato..."


In [51]:
import itertools
from collections import Counter
cooc_counts = Counter()
ing_count  = Counter()
for ingredients in df.ingredients:
    for ing in ingredients:
        ing_count[ing] += 1
    for (ing_a, ing_b) in itertools.combinations(set(ingredients), 2):
       if ing_a > ing_b:
            ing_a, ing_b = ing_b, ing_a
       cooc_counts[(ing_a, ing_b)] += 1

In [52]:
cooc_df = pd.DataFrame(((ing_a, ing_b, ing_count[ing_a], ing_count[ing_b], cooc) for (ing_a, ing_b), cooc in cooc_counts.items()), columns=['a', 'b', 'a_count', 'b_count', 'cooc'])
cooc_df

,a,b,a_count,b_count,cooc
0,milk,white sugar,588,299,61
1,raisins,white sugar,135,299,23
2,eggs,white sugar,874,299,106
3,all-purpose flour,white sugar,1184,299,103
4,baking powder,white sugar,430,299,63
...,...,...,...,...,...
206259,boiling water,white onion,89,214,1
206260,lime wedges,yukon gold potatoes,131,47,1
206261,cilantro sprigs,yukon gold potatoes,64,47,1
206262,boiling water,yukon gold potatoes,89,47,1


In [53]:
cooc_df[cooc_df.a == 'chillies'].sort_values('cooc', ascending=False).head(10)

,a,b,a_count,b_count,cooc
5880,chillies,onions,27,2036,15
5871,chillies,salt,27,4485,13
58761,chillies,garlic,27,1791,9
5867,chillies,garam masala,27,254,9
5883,chillies,vegetable oil,27,1131,8
5878,chillies,tumeric,27,195,7
5879,chillies,cumin seed,27,254,7
26557,chillies,fresh coriander,27,73,7
104656,chillies,ginger,27,435,7
5884,chillies,tomatoes,27,754,7


In [54]:
p_a = cooc_df.a_count / sum(ing_count.values())
p_b = cooc_df.b_count / len(ing_count.values())
p_a_b = cooc_df.cooc / cooc_df.cooc.sum()
cooc_df['pmi'] = np.log(p_a_b / (p_a * p_b))
cooc_df.sort_values('pmi', ascending=False).head(10)

,a,b,a_count,b_count,cooc,pmi
68295,hot dog rolls,pastrami,1,1,1,6.645209
177612,black turtle beans,cured beef,1,1,1,6.645209
168100,crème de menthe,vanilla vodka,1,1,1,6.645209
187683,black treacle,molasses sugar,1,1,1,6.645209
187685,black treacle,clotted cream,1,1,1,6.645209
177487,frozen mango,unsweetened vanilla almond milk,1,1,1,6.645209
103525,nama yuba,uni,1,1,1,6.645209
187693,clotted cream,molasses sugar,1,1,1,6.645209
195679,mccormick taco seasoning,tomato sauce low sodium,1,1,1,6.645209
168533,corn flakes,honey mustard dressing,1,1,1,6.645209


In [55]:
min_count = 5
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count)].sort_values('pmi', ascending=False).head(20)

,a,b,a_count,b_count,cooc,pmi
160792,açai,granola,5,5,3,4.524945
48347,frozen whipped topping,yellow cake mix,5,5,2,4.119480
101586,chia seeds,granola,5,5,2,4.119480
160803,açai,goji berries,5,5,2,4.119480
626,glace cherries,sultana,5,8,3,4.054941
48350,frozen whipped topping,vanilla instant pudding,5,6,2,3.937158
610,glace cherries,mixed spice,5,9,3,3.937158
43697,pepperoncini,roast beef,5,6,2,3.937158
76395,asafoetida powder,split black lentils,12,7,5,3.823830
25074,instant rice,vegetable juice,6,6,2,3.754837


In [56]:
min_count = 30
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count) & (cooc_df.cooc > 1)].sort_values('pmi', ascending=True).head(20)

,a,b,a_count,b_count,cooc,pmi
42127,garlic cloves,white sugar,1535,299,2,-5.698374
5142,fine sea salt,salt,78,4485,2,-5.426847
86081,grated parmesan cheese,ground cumin,481,684,2,-5.365469
20505,garlic cloves,minced garlic,1535,418,4,-5.340264
54930,oil,unsalted butter,456,692,2,-5.323723
41561,sesame oil,tomatoes,410,754,2,-5.303194
41686,cooking spray,soy sauce,364,824,2,-5.272969
30062,salsa,sugar,258,1630,3,-5.205474
94391,all-purpose flour,fresh lime juice,1184,346,3,-5.179272
34409,garlic,large garlic cloves,1791,223,3,-5.153880


In [57]:
from scipy.sparse import csr_matrix
data_df = cooc_df[cooc_df.pmi > 0].copy()
# Since the matrix is symetric, we add the same values for (b,a) as we have for (a,b)
data_df_t = data_df.copy()
data_df.a, data_df.b = data_df.b, data_df.a
data_df = pd.concat([data_df, data_df_t])

rows_idx, row_keys = pd.factorize(data_df.a)
cols_idx, col_keys = pd.factorize(data_df.b)
values = data_df.pmi

matrix = csr_matrix((values, (rows_idx, cols_idx)))
key_to_row = {key: idx for idx, key in enumerate(row_keys)}
matrix

<4462x4150 sparse matrix of type '<class 'numpy.float64'>'
	with 63891 stored elements in Compressed Sparse Row format>

In [58]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(200)
factors = svd.fit_transform(matrix)
factors

array([[ 1.45250650e-03,  3.25485052e-02,  2.36338903e-03, ...,
        -6.86456041e-01,  1.11558067e-02, -7.25113241e-01],
       [ 3.99881088e-03,  3.98521248e-02, -1.49229360e-03, ...,
        -8.82973132e+00,  5.06800602e+00, -7.81914164e+00],
       [ 1.49418445e-02,  6.83329134e-02,  4.38870477e-02, ...,
        -3.51568870e+00,  5.24475528e-02, -4.33118276e+00],
       ...,
       [ 3.82427472e-03,  9.83082161e-03,  2.44399988e-03, ...,
        -1.38399017e-01,  2.09299822e-01, -2.72349057e-01],
       [ 2.06297535e-03,  2.58042297e-03,  3.42825816e-02, ...,
        -5.13882837e-02, -4.84710240e-02,  1.39021279e-02],
       [ 7.70963276e-04,  5.76086959e-04,  5.06002797e-04, ...,
         8.48946356e-02,  2.64447400e-01, -1.34100876e-02]])

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
def most_similar(ingredient, topn=10):
    if ingredient not in key_to_row:
        print("Unknown ingredient.")
    factor = factors[key_to_row[ingredient]]
    cosines = cosine_similarity([factor], factors)[0]
    indices = cosines.argsort()[::-1][:topn + 1]
    keys = [row_keys[idx] for idx in indices if idx != key_to_row[ingredient]]
    return keys, cosines[indices]

def display_most_similar(ingredient, topn=10):
    print("- Most similar to '{}'".format(ingredient))
    for similar_ing, score in zip(*most_similar(ingredient, topn)):
        print("  . {} : {:.2f}".format(similar_ing, score)) 

In [60]:
display_most_similar('chile powder')

- Most similar to 'chile powder'
  . fully cooked luncheon meat : 1.00
  . sliced turkey : 0.74
  . sliced cucumber : 0.73
  . kim chee : 0.70
  . green enchilada sauce : 0.69
  . lite coconut milk : 0.68
  . mexican beer : 0.68
  . baby portobello mushrooms : 0.66
  . marjoram : 0.66
  . nu-salt salt substitute : 0.66


In [61]:
import random

def recommendations(arr):
  final = []
  temp = arr
  final.append(get_recipes(temp))
  for i in range(0, len(arr)):
    initial = temp[i]
    temp2 = most_similar(arr[i])[0]
    for j in temp2:
      temp[i] = j
      final.append(get_recipes(temp))
    temp[i] = initial
  print("Recommended recipes are as follows:")

  for k in (final[0]):
    for j in range(len(df.ingredients)):
      if k == df.ingredients[j]:
        print(df.id[j])

def get_recipes(arr):
  final = []
  for ingredients in df.ingredients:
    flag = 0
    for i in arr:
      if i not in ingredients:
        flag = 1
    if flag == 0:
      final.append(ingredients)
  return final

recommendations(["tomatoes", "thyme"])
#     df.ingredients:
#       print(df.id)
# df.id

# for j in df.ingredients:
#   print(j)

Recommended recipes are as follows:
12741
28088
39487
40229
25666
23323
27838
15163
28933
6293
25351
12935
12132
10358
3726
20270
